In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml import Input
from azure.ai.ml.dsl import pipeline

from utils.login import get_ws_client
from utils.datasets import get_labels_dataset, create_datasets
from utils.computer import create_gpu_cluster
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.entities import AzureBlobDatastore
from azure.ai.ml.entities import AccountKeyConfiguration
from  dotenv import load_dotenv

In [ ]:
from time import sleep
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import (
    BatchEndpoint,
    ModelBatchDeployment,
    ModelBatchDeploymentSettings,
    Model,
    AmlCompute,
    BatchRetrySettings,
    CodeConfiguration,
    Environment,
)
from azure.ai.ml.constants import AssetTypes, BatchDeploymentOutputAction
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
import os

In [ ]:
subscription_id = "b8d6d487-0bd2-4773-b318-12ab763ed178"
resource_group = "strategicai-rg-uks-dev-01"
workspace = "strategicai-mlw-uks-dev-01"

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
DefaultAzureCredential()

In [ ]:
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [ ]:
ml_client

In [ ]:
# https://strategicaistuksdev01.blob.core.windows.net/landing/weekly_refresh_data/
# Set the version number of the data asset (for example: '1')
VERSION = "1"

# Set the path, supported paths include:
# local: './<path>/<file>' (this will be automatically uploaded to cloud storage)
# blob:  'wasbs://<container_name>@<account_name>.blob.core.windows.net/<path>/<file>'
# ADLS gen2: 'abfss://<file_system>@<account_name>.dfs.core.windows.net/<path>/<file>'
# Datastore: 'azureml://datastores/<data_store_name>/paths/<path>/<file>'
path = 'azureml://datastores/csm_data_default/paths/weekly_refresh_data'

In [ ]:
# # Define the Data asset object
# my_data = Data(
#     path=path,
#     type=AssetTypes.URI_FOLDER,
#     description="CSM refresh weekly",
#     name="weekly",
#     version=VERSION,
# )

# # Create the data asset in the workspace
# ml_client.data.create_or_update(my_data)

In [ ]:
import os

dependencies_dir = "./env"
os.makedirs(dependencies_dir, exist_ok=True)

In [ ]:
from azure.ai.ml.entities import Environment
# mcr.microsoft.com/azureml/openmpi5.0-ubuntu24.04:20250601.v1
custom_env_name = "csm-env"
try:
    job_env = ml_client.environments.get(name=custom_env_name,version="7")
except:
    job_env = Environment(
        name=custom_env_name,
        description="Custom environment for csm hpi",
        conda_file=os.path.join(dependencies_dir, "conda.yaml"),
        image="mcr.microsoft.com/azureml/openmpi5.0-ubuntu24.04:20250601.v1",
    )
    job_env = ml_client.environments.create_or_update(job_env)

print(
    f"Environment with name {job_env.name} is registered to workspace, the environment version is {job_env.version}"
)

In [ ]:
# uri = "https://strategicaistuksdev01.blob.core.windows.net/landing/weekly_refresh_data/speaker_exports/BVA24_session_export.csv"
# uri = "https://strategicaistuksdev02.blob.core.windows.net/azureml/landing/weekly_refresh_data/speaker_exports/BVA24_session_export.csv"

In [ ]:
from azure.ai.ml import command, Input, MLClient
from azure.ai.ml.entities import Data, UserIdentityConfiguration, ManagedIdentityConfiguration
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.identity import DefaultAzureCredential

In [ ]:
path = "wasbs://landing@strategicaistuksdev01.blob.core.windows.net/weekly_refresh_data/speaker_exports/BVA24_session_export.csv"
path = "wasbs://azureml@strategicaistuksdev02.blob.core.windows.net/landing/weekly_refresh_data/speaker_exports/BVA24_session_export.csv"

In [ ]:
data_type = AssetTypes.URI_FILE
mode = InputOutputModes.RO_MOUNT

In [ ]:
os.getenv('AZURE_CLIENT_ID')

In [ ]:
identity = ManagedIdentityConfiguration(principal_id=os.getenv('AZURE_CLIENT_ID'))
uri = "azureml://subscriptions/b8d6d487-0bd2-4773-b318-12ab763ed178/resourcegroups/strategicai-rg-uks-dev-01/workspaces/strategicai-mlw-uks-dev-01/datastores/azureml_landing"

# identity = UserIdentityConfiguration()
# Set the input for the job:
inputs = {
    "input_data": "azureml_landing_data"
}
inputs = {
    "uri": uri,
    "post_event_process": "yes",
    "include_previous_scan_data": "no"
}

In [ ]:


job = command(
    code="./prep_dataset",  # local path where the code is stored
    command="python main.py --uri ${{inputs.uri}} --post_event_process ${{inputs.post_event_process}} --include_previous_scan_data ${{inputs.include_previous_scan_data}}",
    inputs=inputs,
    environment=f"{job_env.name}:{job_env.version}",
    compute="cpu-cluster",
    identity=identity,
    is_deterministic=False,
)
    # description,
    # experiment_name


In [ ]:
prep_dataset_yaml = job._to_yaml()

In [ ]:


# Submit the command
ml_client.jobs.create_or_update(job)